In [ ]:
# 기사 풍자 유무 판별 모델
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<a href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/Course%203%20-%20Week%202%20-%20Lesson%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 버전 확인. 
# Run this to ensure TensorFlow 2.x is used
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [2]:
import json
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# 기존 값
# vocab_size = 10000
# embedding_dim = 16
# max_length = 100
# trunc_type='post'
# padding_type='post'
# oov_tok = "<OOV>"
# training_size = 20000

# 하이퍼 파라미터. 얘들을 조정해 모델의 정확도를 설정할 수 있다. 검증 세트에 대한 정확도가 90%가 되는 조합을 찾아보자.
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000


In [ ]:
# 파일 다운로드
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json


In [ ]:
# 파일 열어서 datastore에 담기
with open("/tmp/sarcasm.json", 'r') as f:
    datastore = json.load(f)

sentences = []
labels = []

# 담은 파일들을 나눠서 저장하기(제목, 풍자 유무)
for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])

In [ ]:
# 앞서 설정한 훈련 규모(몇개의 문장까지 받아 토크나이저를 만들 것인가?)를 설정...이 아니라 training_size 기준으로 반갈 하는거다.
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [ ]:
# 토크나이저 생성
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences) # 앞서 training_size만큼 받은 데이터를 이용해 토크나이저 제작

word_index = tokenizer.word_index # 토크나이저 결과

# 문장을 토큰의 연속(시퀀스)로 제작
training_sequences = tokenizer.texts_to_sequences(training_sentences)
# 시퀀스에 패딩 적용. 각 토큰 리스트(시퀀스)의 최대 길이는 max_length고 이거보다 작으면 padding_type에 근거해 0을 채우고 길면 trunc_type에 근거해 토큰을 몇개 버린다. 
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# 위에서 하던거랑 같음. 
testing_sequences = tokenizer.texts_to_sequences(testing_sentences) # testing_sentences : 토크나이저를 따로 만들면 다른 토크나이저가 생성되고(훈련셋에서 자주 나오는 단어랑 테스트셋에서 자주 나오는 단어가 완전히 같을 확률이 100%가 아니니까) 그러면 다른 기준에 의해 시퀀스가 만들어지기 때문에 같은 토크나이저를 써줘야한다. 
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type) # 패딩 적용

In [ ]:
# Need this block to get it to work with TensorFlow 2.x
import numpy as np
# 앞서 준비한 데이터들을 넘파이 배열로 만든다. 모델을 훈련할 때 넘파이 배열을 요구하기 때문이다.
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [ ]:
# 모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length), # 임베딩 레이어. (단어 개수, 차원)
    tf.keras.layers.GlobalAveragePooling1D(), # Flatten()대신 이거 사용. 얘는 평균 풀링을 적용해 일렬로 만든다. 결과값은 별 차이가 없긴 한데 Flatten()이 더 느린 대신 더 정확하고 얘는 더 빠른 대신 더 부정확하다. 근데 별차이 없다. 미세한 차이. 근데 스택 오버플로우에선 Flatten()은 무식한 방법이라며 이거 쓰라고 하더라.
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') # 이진 분류(긍정, 부정)이니까 sigmoid
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
num_epochs = 30
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2) # 훈련

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
# .tsv 출력
# [단어 : 토큰] -> [토큰 : 단어]로 만들기 
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
# 시퀀스를 문장으로
def decode_sentence(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_sentence(training_padded[0]))
print(training_sentences[2])
print(labels[2])

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)


In [ ]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [ ]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [ ]:
sentence = ["granny starting to fear spiders in the garden might be real", "game of thrones season finale showing this sunday night"]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(model.predict(padded))